Runs from uq_score, experiment_sampling, synthetic_data (training), few_shot inference, and step3_score. 

## Global Cells

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install --no-cache-dir git+https://github.com/tlebryk/IE-UQ.git@develop

In [ ]:
from ie_uq import experiment_sampling, synthetic_data, few_shot_inference_extraction
import os 
from doping import step3_score

In [ ]:
import os 
model_id = "meta-llama/Llama-3.2-3B-Instruct"
dataset_path = '/content/drive/MyDrive/nlp/Final_project/IE-UQ/runs/random-100-dataset/train_dataset.jsonl'
inference_dataset_path: str = '/content/drive/MyDrive/nlp/Final_project/IE-UQ/runs/random-100-dataset/train_dataset.jsonl'
mode="synth_json"
output_dir = "/content/drive/MyDrive/nlp/Final_project/IE-UQ/runs"
bnb_dict = {}
peft_dict = None
sft_dict = None
model_dict = None
generation_dict = None
quick_mode=False
budget = 100

synth_json_dir =  os.path.join(output_dir, "synth_json_ft")
synth_span_dir =  os.path.join(output_dir, "synth_span_ft")
extraction_dir =  os.path.join(output_dir, "synth_extraction_ft")


random_baseline_dir = os.path.join(output_dir, "random_baseline")
active_learning_dir = os.path.join(output_dir, "active_learning")
# synth_span_dir = os.path.join(output_dir, "synth_span")

## Sampling (random & active)

In [ ]:
experiment_sampling.main(
    dataset_path=dataset_path,
    sampling_mode="random",
    output_dir=random_baseline_dir,
    budget=budget,
)

In [ ]:
synthetic_data.main(
    model_id=model_id,
    dataset_path=os.path.join(random_baseline_dir, "train_dataset.jsonl"),
    mode="extraction",
    output_dir=random_baseline_dir,
    bnb_dict=bnb_dict, 
    peft_dict=peft_dict,
    sft_dict=sft_dict,
    model_dict=model_dict,
    generation_dict=generation_dict
)

In [ ]:
# few_shot_inference_extraction.main(
#     model_id: str = model_id,
#     dataset_path: str = dataset_path,
#     inference_dataset_path: str = "https://raw.githubusercontent.com/tlebryk/NERRE/refs/heads/main/doping/data/test.json",
#     mode: str = "synth_span",
#     output_dir: str = None,
#     bnb_dict: Optional[Union[str, dict]] = None,
#     # peft_dict: Optional[Union[str, dict]] = None,
#     # sft_dict: Optional[Union[str, dict]] = None,
#     model_dict: Optional[Union[str, dict]] = None,
#     generation_dict: Optional[Union[str, dict]] = None,
#     quick_mode: bool = False,
# )

In [ ]:
step3_score.main(
    # test_file=test_dataset_file,# TODO:
    # pred_file=os.path.join(random_baseline_dir, "fewshot2output.json"),
    plot=True,
    loud=True,
    schema_type=True,
    enforce_lowercase=True,
    output_dir=random_baseline_dir)

In [ ]:
experiment_sampling.main(
    dataset_path=dataset_path,
    sampling_mode="active_learning",
    output_dir=active_learning_dir,
    budget=budget,
)

In [ ]:
synthetic_data.main(
    model_id=model_id,
    dataset_path=os.path.join(active_learning_dir, "train_dataset.jsonl"),
    mode="extraction",
    output_dir=active_learning_dir,
    bnb_dict=bnb_dict, 
    peft_dict=peft_dict,
    sft_dict=sft_dict,
    model_dict=model_dict,
    generation_dict=generation_dict
)

In [ ]:
# TODO FIX
# experiment_sampling.main(
#     dataset_path=dataset_path,
#     sampling_mode="synth_span",
#     output_dir=os.path.join(output_dir, "active_learning"),
#     budget=budget,
# )

## Training
"train_dataset.jsonl"

In [ ]:
# import uq_utils
import torch
import pandas as pd
from tqdm import tqdm
import json
tqdm.pandas()

In [ ]:
# Adjust the path accordingly if your file is located elsewhere
path = '/content/drive/MyDrive/Final_project/NERRE/doping/data/training_json.jsonl'

# Read the JSON file using pandas
# lines=True expects each line to be a valid JSON object
df = pd.read_json(path, lines=True)


In [ ]:
print(df.completion.iloc[0])

 {
 "basemats": {
  "b0": "SrLaMgTaO6"
 },
 "dopants": {
  "d0": "trivalent erbium"
 },
 "dopants2basemats": {
  "d0": [
   "b0"
  ]
 }
}
END


In [ ]:
print(df.prompt.iloc[0].replace("###", ""))

We report the properties of upconversion photoluminescence (UCPL) on SrLaMgTaO6 crystal doped with trivalent erbium at concentrations of 1–20mol% that were synthesized by a solid state reaction method from powder based precursors.

Extract doping information from this sentence.





## Prompting

In [ ]:
def convert_to_chat(prompt, completion, system_prompt = "Extract useful information in a python dictionary from the provided textual data."):
  chat = [
      {"role": "system", "content": system_prompt},
      {"role": "user", "content": prompt},
      {"role": "assistant", "content": completion},
  ]
  return chat

#user_prompt test
prompt = df.prompt.iloc[0].replace("###", "")
# output: 'We report the properties of upconversion photoluminescence (UCPL) on SrLaMgTaO6 crystal doped with trivalent erbium at concentrations of 1–20mol% that were synthesized by a solid state reaction method from powder based precursors.'

#structuredc completion test
completion = df.completion.iloc[0]

# system prompt test (dopant extraction)
#system_prompt = "Extract useful information in a python dictionary from the provided textual data."

In [ ]:
df['chat'] = df.apply(lambda row: convert_to_chat(row['prompt'], row['completion']), axis=1)

## Perplexity

In [ ]:
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig


model_name = "meta-llama/Llama-3.2-1B-Instruct"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

use_flash_attention = False #True if device == torch.device("cuda") else False
# Hugging Face model id
# model_id = "NousResearch/Llama-2-7b-hf"  # non-gated
# model_id = "meta-llama/Llama-2-7b-hf" # gated


# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    # load_in_4bit=True,
    load_in_8bit=True,
    # bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    # quantization_config=bnb_config,
    use_cache=False,
    use_flash_attention_2=use_flash_attention,
    device_map="auto",
    torch_dtype=torch.bfloat16,

)
model.config.pretraining_tp = 1


tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Llama-3.2-1B-Instruct.
403 Client Error. (Request ID: Root=1-6733e5c7-157663151aa3ee911bab7899;d94a85db-5d18-4cdf-9e33-ac7e6ce19aa9)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-3.2-1B-Instruct/resolve/main/config.json.
Access to model meta-llama/Llama-3.2-1B-Instruct is restricted and you are not in the authorized list. Visit https://huggingface.co/meta-llama/Llama-3.2-1B-Instruct to ask for access.

In [ ]:
def calculate_perplexity_1(chat, tokenizer, model):
    # Tokenize the full input
    chat_templated = tokenizer.apply_chat_template(
        chat, tokenize=True,  return_tensors="pt" # add_generation_prompt=True,
    ).to(device)
    # Get the length of the assistant message at the end of the inputs

    outputs = model(chat_templated, labels=chat_templated)
    logits = (
        outputs.logits
    )  # [:, len(prompt_ids[0])-1:-1, :]  # Only completion logits, excluding prompt
    completion_ids = (
        chat_templated  # [:, len(prompt_ids[0]):]  # Only completion token IDs
    )

    # Calculate log probabilities for each token in the completion
    log_probs = torch.nn.functional.log_softmax(logits, dim=-1)
    completion_log_probs = log_probs.gather(2, completion_ids.unsqueeze(-1)).squeeze(-1)
    avg_log_likelihood = -completion_log_probs.mean().item()
    perplexity = torch.exp(torch.tensor(avg_log_likelihood)).item()

    return perplexity

In [ ]:
df['perplexity'] = df.progress_apply(lambda row: calculate_perplexity_raw(row['chat'], tokenizer, model), axis=1)

  0%|          | 1/413 [00:00<00:00, 1735.33it/s]


NameError: name 'tokenizer' is not defined